<img src='https://drive.google.com/uc?export=view&id=1pQINdcWaTfXs2YNqF3_YK2rvOMH247f-' width='600'/>

<img src='https://drive.google.com/uc?export=view&id=1knZ1D-CqkOzm4VrOHFXWRivu0zk0l2D4' width='600'/>

## **목표**

1. RAG 파이프라인을 이해하고 구축한다.
2. RAG 챗봇을 만들 수 있다.
3. Vector Store를 구축할 수 있다.

[전체 시각화 자료 링크](https://www.tldraw.com/r/JaKBU9V2SYg3Zha_vBtlh?v=-3407,-1792,4453,2224&p=QG52B1t54FEiVNBGt-Gzk)

# **1.Pinecone 벡터 DB에 데이터 넣기** (15분)


<img src='https://drive.google.com/uc?export=view&id=14YrhUuSmkf46QOz4mx74Lj64hmCypv1a' width='600'/>


**0번 Vector DB 구축하기**

## **Pinecone 가입 및 API 키 발급**

[노션 가이드 링크](https://www.notion.so/kairos-ku/Pinecone-Setting-69e5f131dbc8474da01be33531c6602e?pvs=4)

## **텍스트 데이터 다운로드**

↓ 고려대에 대한 정보 txt 파일 다운로드 받기


[🔗다운로드 링크](https://drive.google.com/file/d/1ogJJ9zzeTc3yWPNtTOpwlfaxdhXoEiyl/view)


- 고려대학교 챗봇을 만들기 위해 나무위키에서 데이터를 수집함.
- 그냥 복사해서 넣으면 토큰이 너무 큼.

## **텍스트 데이터 쪼개기 (Chunking)**

langchain Splitter를 활용하여 chunk 쪼개기


**< langchain >**

-> LLM 시스템, RAG 시스템 구축과 관련된 코드 제공

-> LLM 프로덕트를 만들 때 가장 많이 활용되는 라이브러리 중 하나

In [ ]:
#langchain,tiktoken 라이브러리 설치
!pip install langchain
!pip install tiktoken

In [ ]:
#파일 불러오기
file_path="./고려대학교 정보.txt"

with open(file_path) as f:
  text = f.read()
text

**< text splitter >**

- 텍스트를 토큰 크기에 따라 조각내는 기능을 가진 클래스

- 1024 토큰마다 텍스트를 chunking

In [ ]:
#text splitter 정의하기
#1024 토큰마다 텍스트를 chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
		chunk_size=1024
)

In [ ]:
#text에 담겨있는 글을 chunking
texts = text_splitter.split_text(text)
len(texts)

<img src='https://drive.google.com/uc?export=view&id=1ZHASGg4jg83qAE0lhO4ryin119JAAkbU' width='600'/>

\

## **텍스트 더미의 Embedding을 구하고 Pinecone에 저장하기**

### **1. pinecone 초기화**

In [ ]:
#라이브러리 설치
!pip install pinecone-client
!pip install tqdm
!pip install openai

In [ ]:
OPENAI_API_KEY = "openai api key 넣기"
PINECONE_API_KEY = "pinecone api key 넣기"

In [ ]:
#pinecone 세팅하기
from pinecone import Pinecone

pc = Pinecone(api_key = PINECONE_API_KEY)
index = pc.Index("data")

### **2. openai 모델을 사용해서 embedding vector 구하기**


In [ ]:
import uuid
from openai import OpenAI
from tqdm import tqdm

# Initialize OpenAI client
client = OpenAI(api_key = OPENAI_API_KEY)

# List to store vectors for upserting
vectors = []


for chunk in texts:
    # Generate embeddings for the chunk
    response = client.embeddings.create(input=chunk, model="text-embedding-3-small")
    chunk_embeddings = response.data[0].embedding

    # Generate a random UUID for the chunk
    chunk_id = str(uuid.uuid4())

    # Create a dictionary for the chunk
    vector_dict = {
        "id": chunk_id,
        "values": chunk_embeddings,
        "metadata": {"chunk": chunk}
      }

    # Add the dictionary to the list
    vectors.append(vector_dict)

<img src='https://drive.google.com/uc?export=view&id=1uH7HRuqVdFcIOQ-EW-kThZmLhUWJvoJg' width='600'/>
<img src='https://drive.google.com/uc?export=view&id=1QZS1LxdsqTh9QFrGZCPUo48qDRO1uCQP' width='600'/>

\

### **3. pinecone에 upsert하기**

- vector DB에서는 insert 대신 upsert라는 표현을 사용

- 모든 데이터를 한번에 보낼 수 없음.

In [ ]:
#10개씩 쪼개서 pinecone에 올리는 코드


batch_size = 10
# Number of batches
num_batches = len(vectors) // 10 + 1

# Iterate over each batch and upsert
for i in range(num_batches):
    # Calculate start and end indices for the current batch
    start_idx = i * batch_size
    end_idx = start_idx + batch_size

    # Slice the vectors list to get the current batch
    batch_vectors = vectors[start_idx:end_idx]

    # Upsert the current batch
    index.upsert(vectors=batch_vectors, namespace="koreauniv")

    # Optional: Print progress
    print(f"Upserted batch {i+1}/{num_batches}")

<img src='https://drive.google.com/uc?export=view&id=13ydxeRzm3jD-nD_5aRUh5pS_TPAYTKbg' width='600'/>

\

# **2. RAG Pipeline 구현 (25분)**

<img src='https://drive.google.com/uc?export=view&id=14YrhUuSmkf46QOz4mx74Lj64hmCypv1a' width='600'/>

\
**1번, 2번 Vector DB 구축하기**

In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install tiktoken
!pip install tqdm
!pip install openai

In [ ]:
OPENAI_API_KEY = "your openai key"
PINECONE_API_KEY = "your pinecone key"

In [ ]:
import uuid
from openai import OpenAI
from tqdm import tqdm
from pinecone import Pinecone


# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)
# Pinecone 이용
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("data")

### **1. Query embedding**

In [ ]:
query = "고려대학교의 전신은 어디야?"
response = client.embeddings.create(input=query, model="text-embedding-3-small")
query_embeddings = response.data[0].embedding

<img src='https://drive.google.com/uc?export=view&id=169VoUNwtFYFcVo2Gz6ErQDetF-_lyGCX' width='600'/>

\

Vector Database에서 query embedding과의 Vector space 거리가 가장 가까운 top 5개의 Chunk를 불러오기 !



In [ ]:
# pinecone의 vector database에서 query와 가까운 답변 가져오기
retrieved_chunks = index.query(
    namespace="koreauniv",
    vector=query_embeddings,
    top_k=5,
    include_values=False,
    include_metadata=True,
)

contexts = ""
for idx,match in enumerate(retrieved_chunks.matches):
    contexts += match["metadata"]["chunk"] + "\n\n"

    # 어떤 chunk가 왔는지 확인해봅시다
    chunk = match["metadata"]["chunk"]
    print(f"chunk{idx+1}: {chunk}\n")

In [ ]:
contexts

### **2. System prompt 재정의**

Contexts 를 참고하여 답변해야 한다고 시스템에 입력을 해주어야 한다.

In [ ]:
messages = [
    {
        "role": "system",
        "content": "당신은 10년차 경력을 가진 고려대학교 투어 챗봇입니다. 대한민국의 고려대학교를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.\n고려대학교에 투어를 온 사람들에게 고려대학교에 대해서 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.\n사람들은 주로 이런 것들을 물어봅니다:\n- 고려대학교의 역사\n- 고려대학교의 건물별 특징\n- 고려대학교의 재밌는 사실들\n\n당신은 사람들에게 다음과 같이 대답해야 합니다:\n- 친절한 말투\n- 항상 존댓말 사용\n- 적절한 이모지 사용\n\n당신은 반드시 제공하는 [Context]에 있는 내용을 기반으로 살을 붙여 답변을 생성해야 합니다."
    }
]

In [ ]:
messages = [
    {
        "role": "system",
        "content": "당신은 10년차 경력을 가진 고려대학교 투어 챗봇입니다. 대한민국의 고려대학교를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.\n고려대학교에 투어를 온 사람들에게 고려대학교에 대해서 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.\n사람들은 주로 이런 것들을 물어봅니다:\n- 고려대학교의 역사\n- 고려대학교의 건물별 특징\n- 고려대학교의 재밌는 사실들\n\n당신은 사람들에게 다음과 같이 대답해야 합니다:\n- 친절한 말투\n- 항상 존댓말 사용\n- 적절한 이모지 사용\n\n당신은 반드시 제공하는 [Context]에 있는 내용을 기반으로 살을 붙여 답변을 생성해야 합니다."
    },
    {
        "role": "system",
        "content": f"[Context]\n{contexts}"
    },
    {
        "role": "user",
        "content": query
    }
]

### **3. Context를 포함시켜 생성**

In [ ]:
result_with_rag = ""
for response in client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=1024,
        stream=True
    ):
    result_with_rag += response.choices[0].delta.content or ""
print(result_with_rag)

<img src='https://drive.google.com/uc?export=view&id=128NvapHHCRyHgulONGQZxdBn8B_u0OGn' width='600'/>

\

## Without RAG

In [ ]:
messages = [
    {
        "role": "system",
        "content": "당신은 10년차 경력을 가진 고려대학교 투어 챗봇입니다. 대한민국의 고려대학교를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.\n고려대학교에 투어를 온 사람들에게 고려대학교에 대해서 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.\n사람들은 주로 이런 것들을 물어봅니다:\n- 고려대학교의 역사\n- 고려대학교의 건물별 특징\n- 고려대학교의 재밌는 사실들\n\n당신은 사람들에게 다음과 같이 대답해야 합니다:\n- 친절한 말투\n- 항상 존댓말 사용\n- 적절한 이모지 사용"
    },
    {
        "role": "user",
        "content": query
    }
]

In [ ]:
result_without_rag = ""
for response in client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=1024,
        stream=True
    ):
    result_without_rag += response.choices[0].delta.content or ""
    print(result_without_rag)

## **RAG 성능 비교해보기**

|질문|필수 포함|
|:---|:---|
|**1.과학도서관에 수영장이 있어?**|수영장x, 과학도서관 설명에 대한 부가 설명|
|**2.고려대학교에서 이용익 선생님의 역할은?**|고려대학교 전신인 보성전문학교 설립||
|**3.입실렌티 어디서 해**|녹지운동장,입실렌티 설명|
|**4.고려대학교 이중전공,융합전공 제도에 대해 알려줘**|이중전공, 융합전공에 대한 정보(이수학점, 학과등)(사실 기반)|
|**5.고려대학교 졸업요건에 대해 알려줘**|130학점이수 및 추가 졸업요건|
|**6.화정체육관에서 e스포츠경기가 열렸어?**|리그오브레전드 결승, 사이버고연전등이 열림|
|**7.고려대학교 캠퍼스 안에 외계인이 방문한 기록이 있어?**|그런거 없음|
|**8.고려대학교 안암병원과 안암역을 연결하는 에스컬레이터는 언제 설치되었어?**|2022년 5월완공|
|**9. 고려대학교 안암병원은 언제 녹지캠퍼스로 이전했어?**|1991년도|

**편하게 비교하기 위해 함수 제공해드립니다!**

In [ ]:
def generate_response(client, model, messages):
    result = ""
    for response in client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.5,
            max_tokens=1024,
            stream=True
        ):
        result += response.choices[0].delta.content or ""
    return result

In [ ]:
def compare_rag_performance(query):
    # Retrieve embeddings for the query
    response = client.embeddings.create(input=query, model="text-embedding-3-small")
    query_embeddings = response.data[0].embedding

    # Retrieve chunks based on the query embeddings
    retrieved_chunks_response = index.query(
        namespace="koreauniv",
        vector=query_embeddings,
        top_k=5,
        include_values=False,
        include_metadata=True,
    )
    retrieved_chunks = [match["metadata"]["chunk"] for match in retrieved_chunks_response.matches]


    # Prepare contexts
    contexts = "\n\n".join(retrieved_chunks)

    # Messages for RAG
    messages_rag = [
        {
            "role": "system",
            "content": "당신은 10년차 경력을 가진 고려대학교 투어 챗봇입니다. 대한민국의 고려대학교를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.\n고려대학교에 투어를 온 사람들에게 고려대학교에 대해서 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.\n사람들은 주로 이런 것들을 물어봅니다:\n- 고려대학교의 역사\n- 고려대학교의 건물별 특징\n- 고려대학교의 재밌는 사실들\n\n당신은 사람들에게 다음과 같이 대답해야 합니다:\n- 친절한 말투\n- 항상 존댓말 사용\n- 적절한 이모지 사용\n\n당신은 반드시 제공하는 [Context]에 있는 내용을 기반으로 살을 붙여 답변을 생성해야 합니다."
        },
        {
            "role": "system",
            "content": f"[Context]\n{contexts}"
        },
        {
            "role": "user",
            "content": query
        }
    ]

    # Messages without RAG
    messages_without_rag = [
        {
            "role": "system",
            "content": "당신은 10년차 경력을 가진 고려대학교 투어 챗봇입니다. 대한민국의 고려대학교를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.\n고려대학교에 투어를 온 사람들에게 고려대학교에 대해서 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.\n사람들은 주로 이런 것들을 물어봅니다:\n- 고려대학교의 역사\n- 고려대학교의 건물별 특징\n- 고려대학교의 재밌는 사실들\n\n당신은 사람들에게 다음과 같이 대답해야 합니다:\n- 친절한 말투\n- 항상 존댓말 사용\n- 적절한 이모지 사용"
        },
        {
            "role": "user",
            "content": query
        }
    ]

    # Generate responses with RAG
    result_rag_gpt35 = generate_response(client, "gpt-3.5-turbo-1106", messages_rag)
    print("Results with RAG (GPT-3.5):", result_rag_gpt35)

    # Generate responses without RAG
    result_without_rag_gpt35 = generate_response(client, "gpt-3.5-turbo-1106", messages_without_rag)
    print("Results without RAG (GPT-3.5):", result_without_rag_gpt35)



In [ ]:
compare_rag_performance("your query")

Results with RAG (GPT-3.5): 안녕하세요! 고려대학교 투어 챗봇입니다. 궁금하신 점이 있으신가요? 고려대학교에 대해 궁금한 것이 있으면 언제든지 물어봐 주세요. 😊

Results without RAG (GPT-3.5): 안녕하세요! 고려대학교 투어 챗봇입니다. 저에게 궁금한 점을 물어보시면 친절히 답변해드리겠습니다. 😊


# **3. RAG streamlit 챗봇 구현** (20분)



In [ ]:
!pip install langchain
!pip install pinecone-client

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install openai

In [ ]:
OPENAI_API_KEY = "your openai key"
PINECONE_API_KEY = "your pinecone key"

In [ ]:
!ngrok authtoken ___입력하기____

In [ ]:
%%writefile app.py
print("hello")

In [ ]:
# app.py 에서 직접 수정하겠습니다.
import streamlit as st
import uuid
from openai import OpenAI
from tqdm import tqdm
from pinecone import Pinecone


OPENAI_API_KEY = "openai api key 넣기"
PINECONE_API_KEY = "pinecone api key 넣기"

st.title("🐯고려대학교 챗봇🐯")
st.write("Hello ☁️🐯KAIROS. 고려대학교에 대한 정보를 물어보세요")

client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("data")

if "openai_model" not in st.session_state:
  st.session_state.openai_model = "gpt-3.5-turbo"

# Initialize chat history
if "messages" not in st.session_state:
  st.session_state.messages = []

system_prompt = {
    "role": "system",
    "content": """
      당신은 10년차 경력을 가진 고려대학교 투어 챗봇입니다.
      대한민국의 고려대학교를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.
      고려대학교에 투어를 온 사람들에게 고려대학교에 대해서 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.
      사람들은 주로 이런 것들을 물어봅니다:
      - 고려대학교의 역사
      - 고려대학교의 건물별 특징
      - 고려대학교의 재밌는 사실들

      당신은 사람들에게 다음과 같이 대답해야 합니다:
      - 친절한 말투
      - 항상 존댓말 사용
      - 적절한 이모지 사용

      당신은 반드시 제공하는 [Context]에 있는 내용을 기반으로 살을 붙여 답변을 생성해야 합니다.
    """
}



# Display chat messages from history on app rerun
for message in st.session_state.messages:
  if message["role"] != "system":
    with st.chat_message(message["role"]):
      st.markdown(message["content"])

# Accept user input for chat
if prompt := st.chat_input("고려대학교에 대해 물어보세요😊"):
  # Add user message to chat history
  st.session_state.messages.append({"role": "user", "content": prompt})

  # Display user message in chat message container
  with st.chat_message("user"):
    st.markdown(prompt)


  # Display assistant response in chat message container
  with st.chat_message("assistant"):
    #입력한 prompt를 embedding
    response = client.embeddings.create(input=prompt, model="text-embedding-3-small")
    query_embeddings = response.data[0].embedding

    # pinecone의 vector database에서 query와 가까운 답변 가져오기
    retrieved_chunks = index.query(
        namespace="koreauniv",
        vector=query_embeddings,
        top_k=5,
        include_values=False,
        include_metadata=True,
    )
    contexts = ""

    for match in retrieved_chunks.matches:
        contexts += match["metadata"]["chunk"] + "\n\n"

    context_prompt = {
        "role" : "system",
        "content" : f"[Context]\n{contexts}"
    }

    messages = [system_prompt, context_prompt]
    messages.extend(st.session_state.messages)

    message_placeholder = st.empty()
    full_response = ""

    for response in client.chat.completions.create(
          model=st.session_state.openai_model,
          messages=messages,
          temperature=0.5,
          max_tokens=1024,
          stream=True
    ):
      full_response += (response.choices[0].delta.content or "")
      message_placeholder.markdown(full_response + "▌")
    message_placeholder.markdown(full_response)
  st.session_state.messages.append({"role": "assistant", "content": full_response})

In [ ]:
from pyngrok import ngrok
!streamlit run app.py&>/dev/null&
publ_url = ngrok.connect(addr='8501')
publ_url

In [ ]:
# streamlit 과 ngrok 종료

In [ ]:
!ps

In [ ]:
!kill <streamlit 번호>

In [ ]:
ngrok.kill()

# **4. 실습 과제** (30분)

- 슈카월드의 10개의 영상을 텍스트로 추출했다.
- 아래의 10가지 질문에 답을 하는 RAG 챗봇을 구현하기